In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from google.colab import files
import matplotlib.pyplot as plt

In [2]:
uploaded = files.upload()

for filename in uploaded.keys():
    df = pd.read_csv(filename)

Saving Train (1).csv to Train (1) (5).csv


In [4]:
df.head()

,Team1,Team2,Stadium,HostCountry,Team1_Venue,Team2_Venue,Team1_Innings,Team2_Innings,MonthOfMatch,MatchWinner
0,5,4,37,4,Home,Away,Second,First,Dec,4
1,1,14,84,7,Neutral,Neutral,First,Second,Sep,1
2,9,15,47,9,Home,Away,First,Second,Feb,9
3,7,2,102,6,Home,Away,First,Second,Aug,2
4,6,8,46,5,Home,Away,First,Second,Aug,6


In [4]:

label_encoders = {}
categ_cols = ['Team1', 'Team2', 'Stadium', 'HostCountry', 'Team1_Venue', 'Team2_Venue', 'Team1_Innings', 'Team2_Innings', 'MonthOfMatch']

for col in categ_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

In [5]:
match_winner_encoder = LabelEncoder()
df['MatchWinner'] = match_winner_encoder.fit_transform(df['MatchWinner'])

In [6]:

X = df.drop('MatchWinner', axis=1)
y = df['MatchWinner']

In [7]:

y = to_categorical(y)

train_size = len(df) - 400
X_train = X.iloc[:train_size]
y_train = y[:train_size]
X_test = X.iloc[train_size:]
y_test = y[train_size:]

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [8]:

model = Sequential()
model.add(Dense(256, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))

In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
history = model.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.2)

Epoch 1/200
53/53 [==============================] - 2s 23ms/step - loss: 2.2013 - accuracy: 0.2533 - val_loss: 2.1334 - val_accuracy: 0.2559
Epoch 2/200
53/53 [==============================] - 1s 18ms/step - loss: 1.9653 - accuracy: 0.3001 - val_loss: 2.0133 - val_accuracy: 0.2962
Epoch 3/200
53/53 [==============================] - 1s 14ms/step - loss: 1.8266 - accuracy: 0.3796 - val_loss: 1.9126 - val_accuracy: 0.3294
Epoch 4/200
53/53 [==============================] - 0s 8ms/step - loss: 1.7080 - accuracy: 0.4009 - val_loss: 1.8908 - val_accuracy: 0.3152
Epoch 5/200
53/53 [==============================] - 0s 8ms/step - loss: 1.5948 - accuracy: 0.4383 - val_loss: 1.8240 - val_accuracy: 0.3365
Epoch 6/200
53/53 [==============================] - 0s 9ms/step - loss: 1.5033 - accuracy: 0.4496 - val_loss: 1.7540 - val_accuracy: 0.3768
Epoch 7/200
53/53 [==============================] - 0s 8ms/step - loss: 1.4672 - accuracy: 0.4526 - val_loss: 1.7695 - val_accuracy: 0.3246
Epoch 8/20

In [11]:
predictions = model.predict(X_test)
predicted_winners = np.argmax(predictions, axis=1)

13/13 [==============================] - 0s 4ms/step


In [12]:
def inverse_transform_column(column, data):
    return label_encoders[column].inverse_transform(data)

In [13]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy}')

13/13 [==============================] - 0s 5ms/step - loss: 3.3022 - accuracy: 0.4850
Accuracy: 0.48500001430511475


In [14]:
team1_encoded = X.iloc[train_size:]['Team1'].values
team2_encoded = X.iloc[train_size:]['Team2'].values
actual_winner_encoded = np.argmax(y_test, axis=1)

In [15]:
team1_decoded = inverse_transform_column('Team1', team1_encoded)
team2_decoded = inverse_transform_column('Team2', team2_encoded)
actual_winner_decoded = match_winner_encoder.inverse_transform(actual_winner_encoded)
predicted_winner_decoded = match_winner_encoder.inverse_transform(predicted_winners)

In [16]:
newdf = pd.DataFrame({
    'Team1': team1_decoded,
    'Team2': team2_decoded,
    'Actual Winner': actual_winner_decoded,
    'Predicted Winner': predicted_winner_decoded
})

In [17]:
print(newdf)

     Team1  Team2  Actual Winner  Predicted Winner
0       14      1              1                14
1        5      4              4                 5
2       12     13             13                12
3        0      3              0                 1
4        6     13             13                13
..     ...    ...            ...               ...
395      5     15              5                 5
396      1     14              1                 1
397     14      5             14                 5
398     15     14             14                13
399      1     10              1                 1

[400 rows x 4 columns]
